# Accessing data from NASA Astrophysics Archives Using Python

NASA missions have collected a lot (XXX) of data spanning a large range (XXX) in wavelengths. These data are housed in four different archives: the HEASARC, MAST, IRSA, NED. However, different physical locations are not an obstacle for astronomers doing multiwavelength science projects that need to make use of data from different archives. The archives have been working together to make sure that you don’t need to know too much about us to get the data you need to work together. We’ve done this by standardizing the way that programs can access the data we house. Since python is a very popular programming language, we are going to use it to show you how you can take advantage of this standardization to put together and analyze a multiwavelength data set. For this workshop, we are using a sample of interacting galaxies, and we will assemble a multiwavelength data set for it. You can generalize this example to much larger data sets of your own.

## The sample: Nearby Interacting Galaxies

In [12]:
from astropy.io import ascii
list_of_galaxies = ascii.read('interacting_gals.csv', delimiter=',')  
print(list_of_galaxies)                       

   ra     dec  
------- -------
9.90704 8.96507
19.0186 46.7304
20.2887 40.4703
125.886 21.3377
136.002 21.9679
141.057 40.6372
  146.7 22.0116
148.785 14.2922
149.751 17.8168
175.039  15.327
    ...     ...
241.519 20.8014
317.088 18.2002
329.235 6.64845
 333.83 37.3012
335.756 30.9249
344.396 19.7827
348.682 4.53407
 350.39 27.1181
350.438 9.07761
357.827 20.5778
0.36165 31.4334
Length = 30 rows


## What does NED know about these galaxies?

In [59]:
import requests
import io
from astropy.table import Table
ned_base_url = 'http://vo.ned.ipac.caltech.edu'
search_radius = 5.0 / 3600.0
for galaxies in list_of_galaxies:
    position = str(galaxies['ra'])+','+str(galaxies['dec'])
    ned_objects_url = ned_base_url + '/services/querySED?REQUEST=queryData&POS='+position+'&SIZE='+str(search_radius)
    ned_objects_response = requests.get(ned_objects_url)
    ned_objects_table = Table.read(io.BytesIO(ned_objects_response.content))
    print(len(ned_objects_table))
    ned_targetname_url = ned_base_url+ned_objects_table[0]['ACREF'].decode("utf-8")
    print(ned_targetname_url)
    ned_data_response = requests.get(ned_targetname_url)
    print(ned_data_response.status_code)
    print(' ')
    ned_data_table = Table.read(io.BytesIO(ned_data_response.content))
    #print(ned_data_table)
    #print(ned_data_table['DataSpectralPassBand'])
    

1
http://vo.ned.ipac.caltech.edu/services/accessSED?REQUEST=getData&TARGETNAME=UGC 00418
200
 
1
http://vo.ned.ipac.caltech.edu/services/accessSED?REQUEST=getData&TARGETNAME=CGCG 551-011
200
 
2
http://vo.ned.ipac.caltech.edu/services/accessSED?REQUEST=getData&TARGETNAME=GALEXASC J012109.21+402816.8
200
 


ValueError: 17:0: no element found